In [ ]:
import pathlib
import shutil
import os

import numpy as np
import pandas as pd

import dbfread

In [ ]:
original_dir = pathlib.Path(r'S:\DataExchange\iViewDB')
decoded_dir = pathlib.Path(r'S:\DataExchange\iViewDB_decoded')

decoded_dir.mkdir(exist_ok=True)

In [ ]:
table_copies_dir = decoded_dir.joinpath('table_copies')
table_copies_dir.mkdir(exist_ok=True)
to_be_indexed_dir = decoded_dir.joinpath('to_be_indexed')
to_be_indexed_dir.mkdir(exist_ok=True)
indexed_dir = decoded_dir.joinpath('indexed')
indexed_dir.mkdir(exist_ok=True)

In [ ]:
original_database_files = list(original_dir.glob('*.DBF')) + list(original_dir.glob('*.MDX')) + list(original_dir.glob('*.DBT'))
for filepath in original_database_files:
    new_path = table_copies_dir.joinpath(filepath.name)
    shutil.copy2(filepath, new_path)

In [ ]:
table_paths = {
    'patient': table_copies_dir.joinpath('PATIENT.DBF'),
    'treatment': table_copies_dir.joinpath('TRTMNT.DBF'),
    'port': table_copies_dir.joinpath('PORT.DBF'),
    'patient_image': table_copies_dir.joinpath('PATIMG.DBF'),
    'frame': table_copies_dir.joinpath('FRAME.DBF'),
}

In [ ]:
table_records = {
    key: list(dbfread.DBF(table_path))
    for key, table_path in table_paths.items()
}

In [ ]:
images_to_be_indexed = list(to_be_indexed_dir.glob('*.jpg'))
file_image_ids = [
    int(path.stem, 16)
    for path in images_to_be_indexed
]

# image_ids

In [ ]:
# images_to_be_indexed

In [ ]:
table_ids = {
    key: [record['DBID'] for record in records]
    for key, records in table_records.items()
}
# table_ids

In [ ]:
pimg_ids = [record['PIMG_DBID'] for record in table_records['frame']]
pimg_id_unique, pimg_id_first_index, pimg_id_counts = np.unique(pimg_ids, return_index=True, return_counts=True)

pimg_id_unique = pimg_id_unique.tolist()
pimg_id_first_index = pimg_id_first_index.tolist()
pimg_id_counts = pimg_id_counts.tolist()

In [ ]:
home = pathlib.Path.home()

In [ ]:
# table_ids['frame']

In [ ]:
for i, encoded_image_path in enumerate(images_to_be_indexed):
    frame_index = table_ids['frame'].index(file_image_ids[i])
    pimg_dbid = table_records['frame'][frame_index]['PIMG_DBID']
    image_data = table_records['patient_image'][table_ids['patient_image'].index(pimg_dbid)]
    port_dpid = image_data['PORT_DBID']

    image_date_time = f"{image_data['IMG_DATE']}_{image_data['IMG_TIME']}"

    machine = image_data['ORG_DTL']
    
    unique_index = pimg_id_unique.index(pimg_dbid)

    first_index = pimg_id_first_index[unique_index]
    total_frames = pimg_id_counts[unique_index]

    frame_number_str_length = len(str(total_frames))
    
    current_frame = frame_index - first_index
    current_frame_str = str(current_frame).zfill(frame_number_str_length)
    
    port_data = table_records['port'][table_ids['port'].index(port_dpid)]

    port_name = port_data['ID']
    treatment_dbid = port_data['TRT_DBID']
    
    treatment_data = table_records['treatment'][table_ids['treatment'].index(treatment_dbid)]

    treatment_name = treatment_data['ID']
    patient_dbid = treatment_data['PAT_DBID']
    
    patient_data = table_records['patient'][table_ids['patient'].index(patient_dbid)]
    patient_dir = f"{patient_data['ID']}_{patient_data['LAST_NAME']}_{patient_data['FIRST_NAME']}"
    
    save_dir = indexed_dir.joinpath(machine, patient_dir, treatment_name, port_name, image_date_time)
    save_dir.mkdir(parents=True, exist_ok=True)
    
    new_path = save_dir.joinpath(f'frame_{current_frame_str}.png')
    
    shutil.copy2(encoded_image_path, home.joinpath('temp_encoded.jpg'))
    
    !bash -c "~/bin/wavelet/jpeg /mnt/c/Users/sbiggs/temp_encoded.jpg /mnt/c/Users/sbiggs/temp_decoded.jpg"
    !bash -c "convert -depth 16 /mnt/c/Users/sbiggs/temp_decoded.jpg /mnt/c/Users/sbiggs/temp_decoded.png"
    
    shutil.copy2(home.joinpath('temp_decoded.png'), new_path)
    encoded_image_path.unlink()